In [ ]:
%reload_ext setup

## Demo

In [ ]:
import asyncio
import os
import logging
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.base import TaskResult


# Suppress autogen_core and httpx INFO logs
logging.getLogger("autogen_core").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

In [ ]:
import json
from pathlib import Path
from igent.tools import read_json

from igent.matching_agent import get_agents
from igent.prompts import load_prompts
from igent.utils import process_pair

# Constants
MAX_ITEMS = 10

async def run_workflow(
    config_path: str,
    registrations_file: str = "registrations.json",
    offers_file: str = "offers.json",
    incentives_file: str = "incentives.json",
    matches_file: str = "matches.json",
    final_matches_file: str = "final_matches.json"
):
    # Load data and prompts
    prompts = await load_prompts()
    registrations = await read_json(registrations_file)
    if not isinstance(registrations, list):
        print("Error: Registrations file must contain a list.")
        return

    max_items = min(MAX_ITEMS, len(registrations))
    if not max_items:
        print("No registrations to process.")
        return

    print(f"Processing {max_items} registrations...")
    offers = await read_json(offers_file)
    incentives = await read_json(incentives_file)

    for i, registration in enumerate(registrations[:max_items], 1):
        registration_id = registration.get("registration_id", "unknown")
        print(f"\nProcessing registration {i}/{max_items} (ID: {registration_id})")

        # Pair 1: Matcher + Critic
        pair1 = await get_agents(
            config_path=config_path,
            matcher_prompt=prompts["a_matcher"],
            critic_prompt=prompts["a_critic"]
        )
        message1 = (
            f"Match registrations to suppliers:\n"
            f"REGISTRATION: ```{[registration]}```\n"
            f"OFFERS: ```{offers}```\n"
            f"On approval, SAVE the output to '{matches_file}' using save_json_tool."
        )
        success1 = await process_pair(pair1, message1, registration_id, "Pair 1", matches_file)
        if not success1:
            print(f"Pair 1 failed for registration {i}. Skipping.")
            continue

        # Pair 2: Subsidy Matcher + Subsidy Critic
        matches = await read_json(matches_file)
        pair2 = await get_agents(
            config_path=config_path,
            matcher_prompt=prompts["b_matcher"],
            critic_prompt=prompts["b_critic"]
        )
        message2 = (
            f"Enrich matches with pricing and subsidies:\n"
            f"MATCHES: ```{matches}```\n"
            f"OFFERS: ```{offers}```\n"
            f"INCENTIVES: ```{incentives}```\n"
            f"On approval, SAVE the output to '{final_matches_file}' using save_json_tool."
        )
        success2 = await process_pair(pair2, message2, registration_id, "Pair 2", final_matches_file)
        if not success2:
            print(f"Pair 2 failed for registration {i}. Continuing.")

    print(f"\nProcessed {max_items} registrations successfully.")

# Run the workflow
await run_workflow(
        config_path="../static/model_config.yaml",
        registrations_file="../data/demo/mock_registrations.json",
        offers_file="../data/demo/mock_offers.json",
        incentives_file="../data/demo/incentives.json",
        matches_file="../data/demo/matches.json",
        final_matches_file="../data/demo/pos.json"
    )

## Sandbox

In [ ]:
%reload_ext setup


In [ ]:
from igent.tools import pop_json

await pop_json(source_file="../data/demo/registrations.json", popped_file="../data/demo/popped.json")

In [ ]:
from igent.tools import update_supplier_capacity
data = """
[  {
    "registration_id": "SPUS52157",
    "campaign_name": "Minnesota 1 Summer 2024",
    "supplier_id": "iSoalr",
    "zip_code": "55409",
    "panel_name": "Silfab HG",
    "panel_capacity": "410",
    "battery_name": null,
    "battery_capacity": null,
    "num_panels": "12",
    "product_type": "Solar",
    "product_price": "15444",
    "eligible_subsidies": [
      "Federal Tax Credit"
    ],
    "payment_type": "Cash"
  }
]
"""
await update_supplier_capacity(data, "../data/demo/offers.json")